In [1]:
# Using the company facts API, we can get a ton of data about many companies
# But tags aren't always the same across companies, and can vary a lot
# Let's analyze this

In [ ]:
import scraper
import pandas as pd

In [3]:
companies = scraper.get_companies()

In [4]:
facts = companies[0].get_facts() # example sets of facts for the first company

In [ ]:
# TODO:
# Analyze the WHOLE of company facts better: not just the us gaap tags
# Look at the DEI taxonomy aswell: it contains some useful things
# Like total number of shares and market cap

In [ ]:
# Notes
    # for some CIK's, you will get request errors when trying to get their company facts
        # This is expected: e.g. an s&p 500 trust filer doesn't have a company facts api
    # The consistency of tags vary considerably!
        # Things like "Assets" are super popular, but consistency drops off steadily
        # For about 90% consistency, you get 42 tags
        # I willl need to decide on a "level of consistency": what tags do i want?
        # Do i want companies with all of those tags? (will be much less)
        # Alt: get tag data individually (though this complicates things considerably)